## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Extract and Map Human phenotype ontology entities to HPO codes

- **Model**: [hpo_mapper_pipeline_en](https://nlp.johnsnowlabs.com/2025/05/02/hpo_mapper_pipeline_en.html)
- **Model Description**: This pipeline is designed to map extracted phenotype entities from clinical or biomedical text to their corresponding Human Phenotype Ontology (HPO) codes. It ensures that observed symptoms, signs, and clinical abnormalities are standardized using HPO terminology.

In [4]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [5]:
import json
import os
import boto3
import pandas as pd
import sagemaker as sage
from sagemaker import ModelPackage
from sagemaker import get_execution_role
from IPython.display import display
from urllib.parse import urlparse


In [6]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [7]:
model_name = "hpo-mapper-pipeline-en"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

## 2. Create a deployable model from the model package.

In [8]:
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)

### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### JSON Format

Provide input as JSON. We support two variations within this format:

1. **Array of Text Documents**: 
   Use an array containing multiple text documents. Each element represents a separate text document.

   ```json
   {
       "text": [
           "Text document 1",
           "Text document 2",
           ...
       ]
   }

    ```

2. **Single Text Document**:
   Provide a single text document as a string.


   ```json
    {
        "text": "Single text document"
    }
   ```

#### JSON Lines (JSONL) Format

Provide input in JSON Lines format, where each line is a JSON object representing a text document.

```
{"text": "Text document 1"}
{"text": "Text document 2"}
```

## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Deploy the SageMaker model to an endpoint

In [9]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

---------!

Once endpoint has been created, you would be able to perform real-time inference.

In [10]:
def invoke_realtime_endpoint(record, content_type="application/json", accept="application/json"):
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=json.dumps(record) if content_type == "application/json" else record,
    )

    response_body = response["Body"].read().decode("utf-8")

    if accept == "application/json":
        return json.loads(response_body)
    elif accept == "application/jsonlines":
        return response_body
    else:
        raise ValueError(f"Unsupported accept type: {accept}")

### Initial Setup

In [11]:
docs = docs = [
    """HPI: A 69-year-old white female with a history of depression, anxiety, admitted to the ABCD Hospital on February 6, 2007, for shortness of breath. The patient was consulted by Psychiatry for anxiety. I know this patient from a previous consult. During this recent admission, she was given Ativan 0.25 mg on a p.r.n. basis with relief after one to two hours. The patient was seen by Abc, MD, and Def, Ph.D. PAST MEDICAL HISTORY: The patient has a history of hypertension, hypothyroidism, cholelithiasis, Port-A-Cath placement, and hydronephrosis.""",
    """A 5-month-old boy brought by his parents because of 2 days of cough. Mother took him when cough started 2 days go to Clinic. But cough got worse and he also started having fever yesterday at night. Mother did not measure it. REVIEW OF SYSTEMS: No vomiting. No diarrhea. No skin rash. No cyanosis."""
]

sample_text = """APNEA: Presumed apnea of prematurity since < 34 wks gestation at birth.
HYPERBILIRUBINEMIA: At risk for hyperbilirubinemia d/t prematurity.
1/25-1/30: Received Amp/Gent while undergoing sepsis evaluation."""

### JSON

In [12]:
input_json_data = {"text": sample_text}
response_json = invoke_realtime_endpoint(input_json_data, content_type="application/json", accept="application/json")
pd.DataFrame(response_json["predictions"][0])

,ner_chunk,begin,end,ner_label,assertion,assertion_confidence,code
0,apnea,16,20,HPO,possible,0.5552,HP:0002104
1,hyperbilirubinemia,91,108,HPO,present,0.8931,HP:0002904
2,sepsis,167,172,HPO,present,0.9881,HP:0100806


### JSON Lines

In [13]:
def create_jsonl(records):
    if isinstance(records, str):
        records = [records]
    json_records = [{"text": text} for text in records]
    json_lines = "\n".join(json.dumps(record) for record in json_records)
    return json_lines

In [14]:
input_jsonl_data = create_jsonl(sample_text)
data = invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": [{"ner_chunk": "apnea", "begin": 16, "end": 20, "ner_label": "HPO", "assertion": "possible", "assertion_confidence": "0.5552", "code": "HP:0002104"}, {"ner_chunk": "hyperbilirubinemia", "begin": 91, "end": 108, "ner_label": "HPO", "assertion": "present", "assertion_confidence": "0.8931", "code": "HP:0002904"}, {"ner_chunk": "sepsis", "begin": 167, "end": 172, "ner_label": "HPO", "assertion": "present", "assertion_confidence": "0.9881", "code": "HP:0100806"}]}


### B. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [15]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [16]:
validation_json_file_name = "input.json"
validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/jsonl/"

def upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/{file_format[1:]}/{file_name}",
        Body=input_data.encode("UTF-8"),
    )

In [17]:
# Create JSON and JSON Lines data
input_jsonl_data = create_jsonl(docs)
input_json_data = json.dumps({"text": docs})

# Upload JSON and JSON Lines data to S3
upload_to_s3(input_json_data, validation_json_file_name)
upload_to_s3(input_jsonl_data, validation_jsonl_file_name)

### JSON

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [19]:
def retrieve_json_output_from_s3(validation_file_name):
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    display(data)

In [20]:
retrieve_json_output_from_s3(validation_json_file_name)

{'predictions': [[{'ner_chunk': 'depression',
    'begin': 38,
    'end': 47,
    'ner_label': 'HPO',
    'assertion': 'present',
    'assertion_confidence': '0.9998',
    'code': 'HP:0000716'},
   {'ner_chunk': 'anxiety',
    'begin': 50,
    'end': 56,
    'ner_label': 'HPO',
    'assertion': 'present',
    'assertion_confidence': '0.9999',
    'code': 'HP:0000739'},
   {'ner_chunk': 'anxiety',
    'begin': 147,
    'end': 153,
    'ner_label': 'HPO',
    'assertion': 'present',
    'assertion_confidence': '0.9894',
    'code': 'HP:0000739'},
   {'ner_chunk': 'hypertension',
    'begin': 332,
    'end': 343,
    'ner_label': 'HPO',
    'assertion': 'present',
    'assertion_confidence': '0.9998',
    'code': 'HP:0000822'},
   {'ner_chunk': 'hypothyroidism',
    'begin': 346,
    'end': 359,
    'ner_label': 'HPO',
    'assertion': 'present',
    'assertion_confidence': '0.9991',
    'code': 'HP:0000821'},
   {'ner_chunk': 'cholelithiasis',
    'begin': 362,
    'end': 375,
    'ner_l

### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [22]:
def retrieve_jsonlines_output_from_s3(validation_file_name):

    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

In [23]:
retrieve_jsonlines_output_from_s3(validation_jsonl_file_name)

{"predictions": [{"ner_chunk": "depression", "begin": 38, "end": 47, "ner_label": "HPO", "assertion": "present", "assertion_confidence": "0.9998", "code": "HP:0000716"}, {"ner_chunk": "anxiety", "begin": 50, "end": 56, "ner_label": "HPO", "assertion": "present", "assertion_confidence": "0.9999", "code": "HP:0000739"}, {"ner_chunk": "anxiety", "begin": 147, "end": 153, "ner_label": "HPO", "assertion": "present", "assertion_confidence": "0.9894", "code": "HP:0000739"}, {"ner_chunk": "hypertension", "begin": 332, "end": 343, "ner_label": "HPO", "assertion": "present", "assertion_confidence": "0.9998", "code": "HP:0000822"}, {"ner_chunk": "hypothyroidism", "begin": 346, "end": 359, "ner_label": "HPO", "assertion": "present", "assertion_confidence": "0.9991", "code": "HP:0000821"}, {"ner_chunk": "cholelithiasis", "begin": 362, "end": 375, "ner_label": "HPO", "assertion": "present", "assertion_confidence": "0.9884", "code": "HP:0001081"}, {"ner_chunk": "hydronephrosis", "begin": 401, "end": 

In [24]:
model.delete_model()

INFO:sagemaker:Deleting model with name: hpo-mapper-pipeline-en-2025-06-09-09-05-17-973


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

